In [18]:
import requests
import pandas as pd

In [19]:
# grab local environmental files (client id, client secret)
import os
from dotenv import load_dotenv
load_dotenv()

True

# Getting all of the albums for my Spotify top 10 Artists (of the month)

First step is going through developer dashboard, getting your client ID and client Secret for use in the client credentials auth flow.

In [3]:
#Auth keys are stored locally in a .env file, to keep hidden.
CL_id = os.getenv('CL_ID')
CL_secret = os.getenv('CL_SECRET')
AUTH_URL = 'https://accounts.spotify.com/api/token'


Next, we're utilizing the client credential flow:
Our first step is going to requesting our access token with the information from the last step.

In [4]:
#client credentials flow

#request authorization
auth_resp = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': CL_id,
    'client_secret': CL_secret,
})

# convert the response to JSON
auth_resp_data = auth_resp.json()

# save the access token
access_token = auth_resp_data['access_token']

We're going to send our requests and include our access token, along with their required Bearer header in a GET request to Spotify's API endpoint

In [14]:
# using the access token

#endpoint to hit 
endpoint = 'https://api.spotify.com/v1/'

#creating the headers
headers = {
  'Authorization': 'Bearer ' + access_token
}

In [11]:
#Helper function to create a dataframe based on the information that we want to receive: artist name, album name, release date, # songs, album cover

def get_album_info(data):
  #values that we're looking for
  artist_name = []
  album_name = []
  release_date = []
  songs_on_album = []
  image_url = []

  #iterate through the json returned to extract the information and append to empty lists
  for album in data['items']:
    artist_name.append(data['items'][0]['artists'][0]['name'])
    album_name.append(album['name'])
    release_date.append(album['release_date'])
    songs_on_album.append(album['total_tracks'])
    image_url.append(album['images'][2]['url'])

  #setting up our dataframe
  album = {
  'artist_name': artist_name,
  'album_name': album_name,
  'release_date': release_date,
  'songs_on_album': songs_on_album,
  'image_url': image_url
  }
  album_df = pd.DataFrame(album)

  return album_df

In [7]:
#creating a function to get endpoint data based on a passed in variable, id

def get_artist(id):
  #sends a GET request to our spotify endpoint with the ID given and our aforementioned header
  r = requests.get(endpoint + 'artists/' + id + '/albums', headers=headers)
  data = r.json()
  return get_album_info(data)

Who are my top ten artists of the month?

In [8]:
#Couldn't get dictionary values to work for some reason
artist_ids = ['0knGpCTbmG4ctl1wzYRZs4','4LLpKhyESsyAXpc4laK94U','5FxD8fkQZ6KcsSYupDVoSO','2h93pZq0e7k5yf4dywlkpM','25uiPmTg16RbhZWAqwLBy5','17Vw9uuOYB7XYjPt0LNFN0','68kEuyFKyqrdQQLLsmiatm','3K2Srho6NCF3o9MswGR76H','6TVVIyd0fsRDGg6WzHKyTP','5RADpgYLOuS2ZxDq7ggYYH']

In [70]:
#ignore, dictionary with artist key, id values
kevs_artists = {
  "Dijon" : "0knGpCTbmG4ctl1wzYRZs4",
  "Mac Miller" : "4LLpKhyESsyAXpc4laK94U",
  "Omar Apollo" : "5FxD8fkQZ6KcsSYupDVoSO",
  "Frank Ocean" : "2h93pZq0e7k5yf4dywlkpM",
  "Charli XCX" : "25uiPmTg16RbhZWAqwLBy5",
  "Machine Girl" : "17Vw9uuOYB7XYjPt0LNFN0",
  "Vince Staples" : "68kEuyFKyqrdQQLLsmiatm",
  "Bakar" : "3K2Srho6NCF3o9MswGR76H",
  "Durand Jones & The Intentions" : "6TVVIyd0fsRDGg6WzHKyTP",
  "Death Grips" : "5RADpgYLOuS2ZxDq7ggYYH",
}

In [12]:
#creating empty dataframe
artist_df = pd.DataFrame()

#iterating through artist_ids list to call get_artist function on all values in the list and creating a new dataframe comprised of everything.
for i in range(len(artist_ids)):
  df = get_artist(artist_ids[i])
  artist_df = pd.concat([artist_df,df])

#let's clean up our albums a little, removing all the singles (where songs on album is 1)
artist_df_albums = artist_df[artist_df['songs_on_album'] > 1] 

In [17]:
artist_df_albums

,artist_name,album_name,release_date,songs_on_album,image_url
0,Dijon,Absolutely,2021-11-05,12,https://i.scdn.co/image/ab67616d00004851a68442...
5,Dijon,How Do You Feel About Getting Married?,2020-05-15,6,https://i.scdn.co/image/ab67616d00004851d060ed...
9,Dijon,Sci Fi 1,2019-03-01,7,https://i.scdn.co/image/ab67616d000048516badb4...
17,Dijon,Letter Blue (Remixes),2022-03-25,4,https://i.scdn.co/image/ab67616d000048510dd482...
19,Dijon,EPHEMERA,2020-10-30,8,https://i.scdn.co/image/ab67616d00004851b426fa...
...,...,...,...,...,...
195,Death Grips,Government Plates,2013-11-13,11,https://i.scdn.co/image/ab67616d00004851dfab0b...
196,Death Grips,No Love Deep Web,2012-10-01,13,https://i.scdn.co/image/ab67616d00004851f552da...
197,Death Grips,The Money Store,2012-04-23,13,https://i.scdn.co/image/ab67616d00004851b4f845...
206,Death Grips,Interview 2016,2016-03-22,6,https://i.scdn.co/image/ab67616d000048513b737b...


In [15]:
artist_df_albums.describe()

,songs_on_album
count,658.000000
mean,13.320669
std,14.043059
min,2.000000
25%,7.000000
50%,12.000000
75%,16.000000
max,100.000000


In [16]:
# converting dataframe to csv file, spotify_albums.csv

artist_df_albums.to_csv(r'spotify_albums.csv',index=False)